In [ ]:
"""
Created on Fri Feb 15 11:43 2021

use script to identify (potentially changing in time) ice shelves in NEMO data from Smith

@author: Clara Burgard
"""

In [ ]:
import xarray as xr
import numpy as np
from pyproj import Transformer
import pandas as pd
from tqdm.notebook import trange, tqdm
import multimelt.plume_functions as pf
import multimelt.box_functions as bf
import multimelt.useful_functions as uf
import multimelt.create_isf_mask_functions as isfmf

from IPython.display import clear_output

In [ ]:
%matplotlib qt5

In [ ]:
map_lim = [-3000000,3000000]

#chunk_size = 700
chunk_size = False

In [ ]:
nemo_run = 'bi646' # 'bf663','bi646'

In [ ]:
######
###### READ IN DATA
######


#if run on luke
inputpath_data='/bettik/burgardc/DATA/NN_PARAM/interim/SMITH_'+nemo_run+'/'
inputpath_metadata='/bettik/burgardc/SCRIPTS/basal_melt_param/data/raw/MASK_METADATA/'
outputpath_mask='/bettik/burgardc/DATA/NN_PARAM/interim/ANTARCTICA_IS_MASKS/SMITH_'+nemo_run+'/'
outputpath_boxes = '/bettik/burgardc/DATA/NN_PARAM/interim/BOXES/SMITH_'+nemo_run+'/'
outputpath_plumes = '/bettik/burgardc/DATA/NN_PARAM/interim/PLUMES/SMITH_'+nemo_run+'/'

file_mask_orig = xr.open_dataset(inputpath_data+'other_mask_vars_Ant_stereo.nc')
file_mask_orig_cut = uf.cut_domain_stereo(file_mask_orig, map_lim, map_lim)

file_mask = xr.open_dataset(inputpath_data+'custom_lsmask_Ant_stereo_clean.nc')#, chunks={'x': chunk_size, 'y': chunk_size})
file_mask_cut = uf.cut_domain_stereo(file_mask, map_lim, map_lim)

file_other = xr.open_dataset(inputpath_data+'corrected_draft_bathy_isf.nc')#, chunks={'x': chunk_size, 'y': chunk_size})
file_other_cut = uf.cut_domain_stereo(file_other, map_lim, map_lim)

file_conc = xr.open_dataset(inputpath_data+'isfdraft_conc_Ant_stereo.nc')
file_conc_cut = uf.cut_domain_stereo(file_conc, map_lim, map_lim)

Create the masks for ice shelves/ground/pinning points/grounding line

In [ ]:
timet = 10
#print('Timestep :'+str(timet.values).zfill(2))

file_TS_orig = xr.open_dataset(inputpath_data + '3D_variables_of_interest_allyy_Ant_stereo_'+str(1970 + timet)+'.nc').isel(time=0).drop('time')
file_TS_cut = uf.cut_domain_stereo(file_TS_orig, map_lim, map_lim)

file_bed_orig = file_mask_orig_cut['bathy_metry'].sel(time=timet).drop('time')
file_draft = file_other_cut['corrected_isfdraft'].sel(time=timet).drop('time')
file_msk = file_mask_cut['ls_mask012'].sel(time=timet).where((file_TS_cut['so'].max('deptht') > 0), 2).drop('time') #0 = ocean, 1 = ice shelves, 2 = grounded ice
file_isf_conc = file_conc_cut['isfdraft_conc'].sel(time=timet).drop('time')

xx = file_mask_cut['x']
yy = file_mask_cut['y']

In [ ]:
whole_ds_tt = isfmf.create_mask_and_metadata_isf(file_msk, -1*file_bed_orig, file_msk, -1*file_draft, file_isf_conc, False, 
                                          '/bettik/burgardc/DATA/NN_PARAM/interim/basins_mask_extrap_50km.nc', outputpath_mask, 
                                          inputpath_metadata + 'iceshelves_metadata_Nico.txt', 
                                          inputpath_metadata+'GL_flux_rignot13.csv', mouginot_basins=True,
                                          write_ismask = 'yes', write_groundmask = 'yes', write_outfile='yes',
                                          ground_point ='no',dist=40, add_fac=120, connectivity=4, threshold=4,
                                          write_metadata='yes')

WITH VARIABLE GEOMETRY

In [ ]:
for tt,timet in enumerate(file_mask_orig_cut.time.sel(time=range(1970,2051))):
    
    clear_output(wait=True)
    
    print('Timestep :'+str(timet.values).zfill(2))

    file_TS_orig = xr.open_dataset(inputpath_data + '3D_variables_of_interest_allyy_Ant_stereo_'+str(timet.values.astype(int))+'.nc').isel(time=0).drop('time')
    file_TS_cut = uf.cut_domain_stereo(file_TS_orig, map_lim, map_lim)

    file_bed_orig = file_mask_orig_cut['bathy_metry'].sel(time=timet).drop('time')
    file_draft = file_other_cut['corrected_isfdraft'].sel(time=timet).drop('time')
    file_msk = file_mask_cut['ls_mask012'].sel(time=timet)#.where((file_TS_cut['so'].max('deptht') > 0), 2).drop('time') #0 = ocean, 1 = ice shelves, 2 = grounded ice
    file_isf_conc = file_conc_cut['isfdraft_conc'].sel(time=timet).drop('time')
    
    xx = file_mask_cut['x']
    yy = file_mask_cut['y']
    
    whole_ds_tt = isfmf.create_mask_and_metadata_isf(file_msk, -1*file_bed_orig, file_msk, -1*file_draft, file_isf_conc, False, 
                                              inputpath_metadata+'lonlat_masks.txt', outputpath_mask, 
                                              inputpath_metadata + 'iceshelves_metadata_Nico.txt', 
                                              inputpath_metadata+'GL_flux_rignot13.csv', mouginot_basins=False, variable_geometry=True,
                                              write_ismask = 'yes', write_groundmask = 'yes', write_outfile='yes',
                                              ground_point ='no',dist=40, add_fac=120, connectivity=4, threshold=4,
                                              write_metadata='yes')

    print('------- WRITE TO NETCDF -----------')
    whole_ds_tt.to_netcdf(outputpath_mask + 'nemo_5km_isf_masks_and_info_and_distance_oneFRIS_'+str(timet.values.astype(int))+'.nc','w')

WITH MOUGINOT BASINS (BUT MISSING SOME PARTS IN VARIABLE GEOMETRY)

In [ ]:
for tt,timet in enumerate(file_mask_orig_cut.time.sel(time=range(1970,2051))):
    
    clear_output(wait=True)
    
    print('Timestep :'+str(timet.values).zfill(2))

    file_TS_orig = xr.open_dataset(inputpath_data + '3D_variables_of_interest_allyy_Ant_stereo_'+str(timet.values.astype(int))+'.nc').isel(time=0).drop('time')
    file_TS_cut = uf.cut_domain_stereo(file_TS_orig, map_lim, map_lim)

    file_bed_orig = file_mask_orig_cut['bathy_metry'].sel(time=timet).drop('time')
    file_draft = file_other_cut['corrected_isfdraft'].sel(time=timet).drop('time')
    file_msk = file_mask_cut['ls_mask012'].sel(time=timet)#.where((file_TS_cut['so'].max('deptht') > 0), 2).drop('time') #0 = ocean, 1 = ice shelves, 2 = grounded ice
    file_isf_conc = file_conc_cut['isfdraft_conc'].sel(time=timet).drop('time')
    
    xx = file_mask_cut['x']
    yy = file_mask_cut['y']
    
    whole_ds_tt = isfmf.create_mask_and_metadata_isf(file_msk, -1*file_bed_orig, file_msk, -1*file_draft, file_isf_conc, False, 
                                              '/bettik/burgardc/DATA/NN_PARAM/interim/basins_mask_extrap_50km.nc', outputpath_mask, 
                                              inputpath_metadata + 'iceshelves_metadata_Nico.txt', 
                                              inputpath_metadata+'GL_flux_rignot13.csv', mouginot_basins=True, variable_geometry=False,
                                              write_ismask = 'yes', write_groundmask = 'yes', write_outfile='yes',
                                              ground_point ='no',dist=40, add_fac=120, connectivity=4, threshold=4,
                                              write_metadata='yes')

    print('------- WRITE TO NETCDF -----------')
    whole_ds_tt.to_netcdf(outputpath_mask + 'nemo_5km_isf_masks_and_info_and_distance_oneFRIS_'+str(timet.values.astype(int))+'.nc','w')

Prepare the box characteristics (writes the output directly to files)

In [ ]:

for tt,timet in enumerate(file_mask_orig_cut.time.sel(time=range(1970,2051))): 

    clear_output(wait=True)
    
    print('Timestep :'+str(timet.values).zfill(2))
    file_bed_orig = file_mask_orig_cut['bathy_metry'].sel(time=timet).drop('time')
    file_draft = file_other_cut['corrected_isfdraft'].sel(time=timet).drop('time')
    file_msk = file_mask_cut['ls_mask012'].sel(time=timet)#.where((file_TS_cut['so'].max('deptht') > 0), 2).drop('time') #0 = ocean, 1 = ice shelves, 2 = grounded ice
    file_isf_conc = file_conc_cut['isfdraft_conc'].sel(time=timet).drop('time')

    xx = file_mask_cut['x']
    yy = file_mask_cut['y']

    whole_ds_tt = xr.open_dataset(outputpath_mask + 'nemo_5km_isf_masks_and_info_and_distance_oneFRIS_'+str(timet.values.astype(int))+'.nc')

    nonnan_Nisf = whole_ds_tt['Nisf'].where(np.isfinite(whole_ds_tt['front_bot_depth_max']), drop=True).astype(int)
    file_isf_nonnan = whole_ds_tt.sel(Nisf=nonnan_Nisf)
    large_isf = file_isf_nonnan['Nisf'].where(file_isf_nonnan['isf_area_here'] >= 2500, drop=True)
    file_isf = file_isf_nonnan.sel(Nisf=large_isf)
    if 'labels' in file_isf.coords.keys():
        file_isf = file_isf.drop('labels')

    isf_var_of_int = whole_ds_tt[['ISF_mask', 'GL_mask', 'dGL', 'dIF', 'latitude', 'longitude', 'isf_name']]
    out_2D, out_1D = bf.box_charac_file(file_isf['Nisf'],isf_var_of_int, -1*file_draft, file_isf_conc, outputpath_boxes, max_nb_box=10)
    out_2D.to_netcdf(outputpath_boxes + 'nemo_5km_boxes_2D_oneFRIS_'+str(timet.values.astype(int))+'.nc')
    out_1D.to_netcdf(outputpath_boxes + 'nemo_5km_boxes_1D_oneFRIS_'+str(timet.values.astype(int))+'.nc')
    
    

FOR WILKINS AND BACH (BECAUSE HAD A PROBLEM IN FIRST PREPROCESSING)

In [ ]:

for tt,timet in enumerate(file_mask_orig_cut.time.sel(time=range(1970,2051))):

    clear_output(wait=True)
    
    print('Timestep :'+str(timet.values).zfill(2))
    file_bed_orig = file_mask_orig_cut['bathy_metry'].sel(time=timet).drop('time')
    file_draft = file_other_cut['corrected_isfdraft'].sel(time=timet).drop('time')
    file_msk = file_mask_cut['ls_mask012'].sel(time=timet)#.where((file_TS_cut['so'].max('deptht') > 0), 2).drop('time') #0 = ocean, 1 = ice shelves, 2 = grounded ice
    file_isf_conc = file_conc_cut['isfdraft_conc'].sel(time=timet).drop('time')

    xx = file_mask_cut['x']
    yy = file_mask_cut['y']

    whole_ds_tt = xr.open_dataset(outputpath_mask + 'nemo_5km_isf_masks_and_info_and_distance_oneFRIS_'+str(timet.values.astype(int))+'.nc')
    
    nisf_list = []
    nonnan_Nisf = whole_ds_tt['Nisf'].where(np.isfinite(whole_ds_tt['front_bot_depth_max']), drop=True).astype(int)
    file_isf_nonnan = whole_ds_tt.sel(Nisf=nonnan_Nisf)
    large_isf = file_isf_nonnan['Nisf'].where(file_isf_nonnan['isf_area_here'] >= 2500, drop=True)
    file_isf = file_isf_nonnan.sel(Nisf=large_isf)
    if 'labels' in file_isf.coords.keys():
        file_isf = file_isf.drop('labels')
    if 75 in large_isf:
        nisf_list.append(75)
    if 54 in file_isf.Nisf:
        nisf_list.append(54)

    isf_var_of_int = whole_ds_tt[['ISF_mask', 'GL_mask', 'dGL', 'dIF', 'latitude', 'longitude', 'isf_name']]
    out_2D, out_1D = bf.box_charac_file(file_isf['Nisf'].sel(Nisf=nisf_list),isf_var_of_int, -1*file_draft, file_isf_conc, outputpath_boxes, max_nb_box=10)
    out_2D.to_netcdf(outputpath_boxes + 'nemo_5km_boxes_2D_oneFRIS_'+str(timet.values.astype(int))+'_only_AlexIsland.nc')
    out_1D.to_netcdf(outputpath_boxes + 'nemo_5km_boxes_1D_oneFRIS_'+str(timet.values.astype(int))+'_only_AlexIsland.nc')
    
    

In [ ]:
for timet in tqdm(file_mask_orig_cut.time.sel(time=range(1970,2051))): 
    #print(timet.values)
    file_isf = xr.open_dataset(outputpath_mask + 'nemo_5km_isf_masks_and_info_and_distance_oneFRIS_'+str(timet.values.astype(int))+'.nc')
    out_2D_AI = xr.open_dataset(outputpath_boxes + 'nemo_5km_boxes_2D_oneFRIS_'+str(timet.values.astype(int))+'_only_AlexIsland.nc')
    out_1D_AI = xr.open_dataset(outputpath_boxes + 'nemo_5km_boxes_1D_oneFRIS_'+str(timet.values.astype(int))+'_only_AlexIsland.nc')
    out_2D_orig = xr.open_dataset(outputpath_boxes + 'nemo_5km_boxes_2D_oneFRIS_'+str(timet.values.astype(int))+'.nc')
    out_1D_orig = xr.open_dataset(outputpath_boxes + 'nemo_5km_boxes_1D_oneFRIS_'+str(timet.values.astype(int))+'.nc')

    merged_2D = out_2D_orig.where(file_isf['ISF_mask'] != 75, out_2D_AI).load()
    merged_1D = xr.concat([out_1D_orig.drop_sel(Nisf=75),out_1D_AI], dim='Nisf').load()

    merged_2D.to_netcdf(outputpath_boxes + 'nemo_5km_boxes_2D_oneFRIS_'+str(timet.values.astype(int))+'_merged75.nc', 'w')
    merged_1D.to_netcdf(outputpath_boxes + 'nemo_5km_boxes_1D_oneFRIS_'+str(timet.values.astype(int))+'_merged75.nc', 'w')


Prepare the plume characteristics

In [ ]:
plume_param_options = ['simple','lazero', 'appenB']


for tt,timet in enumerate(file_mask_orig_cut.time.sel(time=range(1970,2051))): 

    clear_output(wait=True)
    
    print('Timestep :'+str(timet.values).zfill(2))
    file_draft = file_other_cut['corrected_isfdraft'].sel(time=timet).drop('time')

    xx = file_mask_cut['x']
    yy = file_mask_cut['y']

    whole_ds_tt = xr.open_dataset(outputpath_mask + 'nemo_5km_isf_masks_and_info_and_distance_oneFRIS_'+str(timet.values.astype(int))+'.nc')

    nonnan_Nisf = whole_ds_tt['Nisf'].where(np.isfinite(whole_ds_tt['front_bot_depth_max']), drop=True).astype(int)
    file_isf_nonnan = whole_ds_tt.sel(Nisf=nonnan_Nisf)
    large_isf = file_isf_nonnan['Nisf'].where(file_isf_nonnan['isf_area_here'] >= 2500, drop=True)
    file_isf = file_isf_nonnan.sel(Nisf=large_isf)
    if 'labels' in file_isf.coords.keys():
        file_isf = file_isf.drop('labels')

    plume_var_of_int = file_isf[['ISF_mask', 'GL_mask', 'IF_mask', 'dIF', 'dGL_dIF', 'latitude', 'longitude', 'front_ice_depth_avg']]

    # Compute the ice draft
    ice_draft_pos = file_draft
    ice_draft_neg = -1*ice_draft_pos


    plume_charac = pf.prepare_plume_charac(plume_param_options, ice_draft_pos, plume_var_of_int)
    print('------ WRITE TO NETCDF -------')
    plume_charac.to_netcdf(outputpath_plumes+'nemo_5km_plume_characteristics_oneFRIS_'+str(timet.values.astype(int))+'.nc')

ONLY WILKINS AND BACH

In [ ]:
plume_param_options = ['simple','lazero', 'appenB']


for tt,timet in enumerate(file_mask_orig_cut.time.sel(time=range(1970,2051))): 

    clear_output(wait=True)
    
    print('Timestep :'+str(timet.values).zfill(2))
    file_draft = file_other_cut['corrected_isfdraft'].sel(time=timet).drop('time')

    xx = file_mask_cut['x']
    yy = file_mask_cut['y']

    whole_ds_tt = xr.open_dataset(outputpath_mask + 'nemo_5km_isf_masks_and_info_and_distance_oneFRIS_'+str(timet.values.astype(int))+'.nc')
    
    nisf_list = []
    nonnan_Nisf = whole_ds_tt['Nisf'].where(np.isfinite(whole_ds_tt['front_bot_depth_max']), drop=True).astype(int)
    file_isf_nonnan = whole_ds_tt.sel(Nisf=nonnan_Nisf)
    large_isf = file_isf_nonnan['Nisf'].where(file_isf_nonnan['isf_area_here'] >= 2500, drop=True)
    file_isf = file_isf_nonnan.sel(Nisf=large_isf)
    if 'labels' in file_isf.coords.keys():
        file_isf = file_isf.drop('labels')
    if 75 in large_isf:
        nisf_list.append(75)
    if 54 in file_isf.Nisf:
        nisf_list.append(54)
    file_isf = file_isf.sel(Nisf=nisf_list)

    plume_var_of_int = file_isf[['ISF_mask', 'GL_mask', 'IF_mask', 'dIF', 'dGL_dIF', 'latitude', 'longitude', 'front_ice_depth_avg']]

    # Compute the ice draft
    ice_draft_pos = file_draft
    ice_draft_neg = -1*ice_draft_pos


    plume_charac = pf.prepare_plume_charac(plume_param_options, ice_draft_pos, plume_var_of_int)
    print('------ WRITE TO NETCDF -------')
    #plume_charac.to_netcdf(outputpath_plumes+'nemo_5km_plume_characteristics.nc')
    plume_charac.to_netcdf(outputpath_plumes+'nemo_5km_plume_characteristics_oneFRIS_'+str(timet.values.astype(int))+'_only_AlexIsland.nc')

In [ ]:
for timet in file_mask_orig_cut.time.sel(time=range(1970,2051)): # continue at 1983
    print(timet.values)
    file_isf = xr.open_dataset(outputpath_mask + 'nemo_5km_isf_masks_and_info_and_distance_oneFRIS_'+str(timet.values.astype(int))+'.nc')
    plume_AI = xr.open_dataset(outputpath_plumes+'nemo_5km_plume_characteristics_oneFRIS_'+str(timet.values.astype(int))+'_only_AlexIsland.nc')
    plume_orig = xr.open_dataset(outputpath_plumes+'nemo_5km_plume_characteristics_oneFRIS_'+str(timet.values.astype(int))+'.nc')

    merged_plume = plume_orig.where(file_isf['ISF_mask'] != 75, plume_AI)

    merged_plume.to_netcdf(outputpath_plumes+'nemo_5km_plume_characteristics_oneFRIS_'+str(timet.values.astype(int))+'_merged75.nc', 'w')


Prepare correct bathymetry (accounting for ice shelf concentration but also if we are at ice front or grounding line)

In [ ]:
whole_ds = xr.open_dataset(outputpath_mask + 'nemo_5km_isf_masks_and_info_and_distance_new_oneFRIS.nc')

file_bed_orig = file_mask_orig_cut['bathy_metry']
file_bed_corr = file_other_cut['corrected_isf_bathy']
file_draft = file_other_cut['corrected_isfdraft'] 

file_bed_goodGL = file_bed_orig.where(file_draft < file_bed_orig,file_bed_corr)
file_bed_goodGL_with_ocean =  file_bed_goodGL.where(whole_ds['ISF_mask'] > 1, file_bed_orig)
file_bed_goodGL_with_ocean.to_dataset(name='bathymetry').to_netcdf(outputpath_mask + 'processed_bathymetry.nc')